In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
dsf = pd.read_csv('./movies_final.csv')

In [3]:
dsf

,Movie,OMDB,Year,Runtime,Genres,Certificate,Rating,Votes,Director,Stars,Budget,BoxOffice,BoxOfficeWorldwide,RoI,RoI_WW
0,Thugs of Hindostan,tt5970844,2018,0,Action|Adventure,No,0.0,0,Vijay Krishna Acharya,Aamir Khan|Katrina Kaif|Amitabh Bachchan|Fatim...,0.0,0.0,0.0,0.000000,0.000000
1,Batti Gul Meter Chalu,tt7720922,2018,161,Drama,PG,6.3,1552,Shree Narayan Singh,Shahid Kapoor|Shraddha Kapoor|Divyendu Sharma|...,50.0,37.0,37.0,0.740000,0.740000
2,Dangal,tt5074352,2016,161,Action|Biography|Drama,PG,8.5,105480,Nitesh Tiwari,Aamir Khan|Sakshi Tanwar|Fatima Sana Shaikh|Sa...,90.0,370.0,2000.0,4.111111,22.222222
3,Manmarziyaan,tt5474036,2018,156,Romance,12A,7.0,1836,Anurag Kashyap,Tapsee Pannu|Vicky Kaushal|Abdul Quadir Amin|A...,30.0,26.0,28.0,0.866667,0.933333
4,Sui Dhaaga: Made in India,tt7147540,2018,122,Comedy|Drama,PG,6.9,1342,Sharat Katariya,Anushka Sharma|Varun Dhawan|Raghuvir Yadav|Nam...,50.0,79.0,125.0,1.580000,2.500000
5,Stree,tt8108202,2018,128,Comedy|Horror,15,8.3,7801,Amar Kaushik,Rajkummar Rao|Shraddha Kapoor|Pankaj Tripathi|...,25.0,127.0,180.0,5.080000,7.200000
6,Manto,tt6923462,2018,112,Biography|Drama,UA,7.9,622,Nandita Das,Nawazuddin Siddiqui|Rasika Dugal|Tahir Raj Bha...,8.0,3.5,3.7,0.437500,0.462500
7,Pataakha,tt8396128,2018,134,Action|Comedy|Drama,12A,7.7,392,Vishal Bhardwaj,Sanya Malhotra|Radhika Madan|Sunil Grover|Vija...,20.0,6.5,7.0,0.325000,0.350000
8,3 Idiots,tt1187043,2009,170,Comedy|Drama,12A,8.4,282358,Rajkumar Hirani,Aamir Khan|Madhavan|Mona Singh|Sharman Joshi,55.0,202.0,392.0,3.672727,7.127273
9,Slumdog Millionaire,tt1010048,2008,120,Drama|Romance,15,8.0,723061,Danny Boyle,Loveleen Tandan|Dev Patel|Freida Pinto|Saurabh...,0.0,0.0,0.0,0.000000,0.000000


In [6]:
ds = dsf[dsf.Budget != 0]

In [7]:
print(dsf.shape)
print(ds.shape)

(4919, 15)
(990, 15)


In [8]:
np.mean(dsf.Budget)

5.100670868062615

In [11]:
np.mean(dsf.BoxOffice)

6.306354950193128

In [14]:
ds.dtypes

Movie                  object
OMDB                   object
Year                    int64
Runtime                 int64
Genres                 object
Certificate            object
Rating                float64
Votes                   int64
Director               object
Stars                  object
Budget                float64
BoxOffice             float64
BoxOfficeWorldwide    float64
RoI                   float64
RoI_WW                float64
dtype: object

In [12]:
genre = ds.Genres.str.split('|', expand=True).stack()
star = ds.Stars.str.split(r'|', expand=True).stack().reset_index(level=1, drop=True).to_frame('Stars')
cert = ds.Certificate.str.split('|', expand=True).stack()
direc = ds.Director.str.split('|', expand=True).stack()

In [13]:
movie = ds.Movie
year = ds.Year
run = ds.Runtime
rating = ds.Rating
vote = ds.Votes
budget = ds.Budget
boxoffice = ds.BoxOffice
roi = ds.RoI

In [15]:
df_genre = pd.get_dummies(genre, prefix='g').groupby(level=0).sum()
df_star = pd.get_dummies(star, prefix='s', columns=['Stars']).groupby(level=0).sum()
df_cert = pd.get_dummies(cert, prefix='c').groupby(level=0).sum()
df_dir = pd.get_dummies(direc, prefix='d').groupby(level=0).sum()
df_year = pd.get_dummies(year, prefix='y').groupby(level=0).sum()

In [16]:
runtime = []
for r in run:
    runtime.append(int(r))
runtime = np.array(runtime)
runtime_cat = np.zeros(runtime.shape)
df_rt = pd.DataFrame(runtime, index=run.index ,columns = ['Runtime'])

rt_bins = [-1, 90, 120, 150, 180, 360]
rt_labels = [1, 2, 3, 4, 5]
df_rt['Runtime_Bins'] = pd.cut(df_rt['Runtime'], bins=rt_bins)
#print(df_rt)
run_bins = df_rt['Runtime_Bins']

df_runtime = pd.get_dummies(run_bins, prefix='rt').groupby(level=0).sum()

In [19]:
rtg_bins = [0, 2, 4, 6, 8, 10]
ratings = []
for r in rating:
    ratings.append(float(r))
ratings = np.array(ratings)
ratings_cat = np.zeros(ratings.shape)
df_rtg = pd.DataFrame(ratings, index=rating.index, columns=['Rating'])

df_rtg['Rating_Bins'] = pd.cut(df_rtg['Rating'], bins=rtg_bins)
rat_bins = df_rtg['Rating_Bins']

df_rating = pd.get_dummies(rat_bins, prefix='rat').groupby(level=0).sum()

In [22]:
df_roi = pd.DataFrame(roi)
roi_bins = [0.0, 0.3, 0.8, 1.0, 1.5, 2.5, 10]
roi_labels = [1, 2, 3, 4, 5, 6]
df_roi['Target'] = pd.cut(df_roi['RoI'], bins=roi_bins, labels=roi_labels)
df_roiclasses = df_roi['Target']
df_y = pd.get_dummies(df_roi['Target'], prefix='roi').groupby(level=0).sum()

In [25]:
final_df = pd.concat([movie, df_roiclasses, budget, boxoffice, rating, vote,  year, df_rating, df_genre, df_year, df_runtime, df_dir, df_star], axis=1)

In [29]:
final_df.to_csv('movies_prerelease_full.csv', encoding='utf-8', index=False)

In [31]:
year_df = pd.concat([movie, df_roiclasses, budget, boxoffice, rating, vote,  year, df_year], axis=1)
year_df.to_csv('movies_prerelease_year.csv', encoding='utf-8', index=False)

In [34]:
genre_df = pd.concat([movie, df_roiclasses, budget, boxoffice, rating, vote, df_genre], axis=1)
genre_df.to_csv('movies_prerelease_genre.csv', encoding='utf-8', index=False)

In [35]:
yeargenre_df = pd.concat([movie, df_roiclasses, budget, boxoffice, rating, vote, year, df_year, df_genre], axis=1)
yeargenre_df.to_csv('movies_prerelease_yeargenre.csv', encoding='utf-8', index=False)

In [36]:
final2_df = pd.concat([movie, df_roiclasses, budget, boxoffice, rating, vote, df_dir, df_star], axis=1)
final2_df.to_csv('movies_prerelease_fullcast.csv', encoding='utf-8', index=False)